In [1]:
import xarray as xr
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import glob
from pylab import shape,size
import pandas as pd
import yaml
from datetime import date

In [2]:
data_dir = '../Data/'
flight_seg_dir = '../joanne/flight_segments/'
sondes_yaml_dir = '../joanne/Level_2/'

levels_dir = np.empty(5,dtype='U16')

for i in range(5) :
    levels_dir[i] = data_dir+f'Level_{i}/'

## Flight Segment files

In [3]:
fs_files = sorted(glob.glob(flight_seg_dir+'*.yaml'))
sondes_yaml = yaml.load(open(sondes_yaml_dir+'sondes_for_flightphase.yaml'),Loader=yaml.SafeLoader)

In [77]:
circle_times = []
flight_date = []
takeoff = {}
landing = {}
circles_in_flight = {}
platform_name = []
segment_id = []
circle_count = {}

for i in fs_files:

    with open(i) as source:
        flightinfo = yaml.load(source, Loader=yaml.SafeLoader)

    circle_count[flightinfo['flight_id']] = [c["segment_id"]
            for c in flightinfo["segments"]
            if "circle" in c["kinds"]]
            # if len(c["dropsondes"]["GOOD"]) >= 6

    segment_id.append(
        [
            (c["segment_id"])
            for c in flightinfo["segments"]
            if "circle" in c["kinds"]
            if len(c["dropsondes"]["GOOD"]) >= 6
        ]
    )

    if "HALO" in i:
        platform_name.append("HALO")
    elif "P3" in i:
        platform_name.append("P3")
    else:
        platform_name.append("")

    flight_date.append(np.datetime64(date.strftime(flightinfo["date"], "%Y-%m-%d")))
    takeoff[flightinfo['flight_id']] = np.datetime64(date.strftime(flightinfo["takeoff"], "%Y-%m-%d %H:%M:%S"))
    landing[flightinfo['flight_id']] = np.datetime64(date.strftime(flightinfo["landing"], "%Y-%m-%d %H:%M:%S"))
    circles_in_flight[flightinfo['flight_id']] = int(len(circle_count[flightinfo['flight_id']]))

In [283]:
df_flights = [None] * len(fs_files)

for id_,i in enumerate(fs_files):

    with open(i, 'r') as f:
        segments = yaml.load(f)['segments']
        df_flights[id_] = pd.io.json.json_normalize([c for c in segments if 'circle' in c['kinds']])

    if len(df_flights[id_]) != 0 :
        total_sondes = [None] * len(df_flights[id_]['kinds'])

        for g in range(len(df_flights[id_]['kinds'])):
            total_sondes[g] = len(df_flights[id_]['dropsondes.GOOD'][g]) + len(df_flights[id_]['dropsondes.BAD'][g]) + len(df_flights[id_]['dropsondes.UGLY'][g])

        df_flights[id_]['total_sondes'] = pd.Series(total_sondes)

        drop_list = []
        for g in range(len(total_sondes)) :
            if total_sondes[g] == 0 :
                print(id_,g)
                drop_list.append(g)
        
        if len(drop_list) > 0 :
            df_flights[id_].drop(drop_list,inplace=True)

3 0
3 1
3 2
24 3


In [306]:
circles_df_flights = pd.concat([i for i in df_flights if len(i) != 0])

## Level-4 Dataset

In [289]:
l4_file = sorted(glob.glob(levels_dir[4]+'*v0.8.1+11*.nc'))
l4 = xr.open_dataset(l4_file[-1])

In [340]:
circles_table = pd.DataFrame(index=pd.Series(l4.segment_id.values,name='Circle ID'),data={'Circle Time':pd.DatetimeIndex(l4.circle_time.values).strftime('%H:%M'),'Longitude':l4.circle_lon.values,'Latitude':l4.circle_lat.values,'Diameter':l4.circle_diameter.values/1000,'Dropsondes' : circles_df_flights['total_sondes'].values})#,'Minimum Sondes Regressed':l4.sondes_regressed.where(l4.sondes_regressed!=0,drop=True).min(dim='alt').astype(int)})

caption = 'Details of circles flown during \EURECA. Time is the mean launch time for all sondes in the circle. Longitude (\si{\degree}E), latitude (\si{\degree}N) and diameter (\si{\kilo\meter}) are of a least-squares fitted circle to all sondes. Dropsondes show total number of sondes launched in the circle. The sondes used for regression may vary as some sondes might be faulty. The sondes regressed at every altitude level is available as a variable in Level-4.'

print(circles_table.reset_index().to_latex(caption=caption,label='tab:circle_sondes',column_format='cccccc',index=False,longtable=True,float_format="%.2f"))


\begin{longtable}{cccccc}
\caption{Details of circles flown during \EURECA. Time is the mean launch time for all sondes in the circle. Longitude (\si{\degree}E), latitude (\si{\degree}N) and diameter (\si{\kilo\meter}) are of a least-squares fitted circle to all sondes. Dropsondes show total number of sondes launched in the circle. The sondes used for regression may vary as some sondes might be faulty. The sondes regressed at every altitude level is available as a variable in Level-4.}\label{tab:circle_sondes}\\
\toprule
    Circle ID & Circle Time &  Longitude &  Latitude &  Diameter &  Dropsondes \\
\midrule
\endhead
\midrule
\multicolumn{6}{r}{{Continued on next page}} \\
\midrule
\endfoot

\bottomrule
\endlastfoot
  P3-0117\_ci1 &       15:55 &     -51.00 &     14.84 &    181.83 &          12 \\
  P3-0119\_ci1 &       15:02 &     -52.97 &     14.50 &    180.94 &          12 \\
  P3-0123\_ci1 &       14:32 &     -54.96 &     14.38 &    186.96 &          12 \\
  P3-0123\_ci2 &       

In [6]:
with open(sondes_yaml_dir+'sondes_for_flightphase.yaml', 'r') as f:
    df = pd.io.json.json_normalize(yaml.load(f))

df['flight_id'] = [df['sonde_id'][m][:-4] for m in range(len(df['sonde_id']))] 

sondes_per_flight = pd.DataFrame(df.groupby(['flight_id']).count()['sonde_id'])
sondes_per_flight.rename(columns={'sonde_id':'Dropsondes launched'},inplace=True)
sondes_per_flight.index.rename('Flight ID',inplace=True)
sondes_per_flight['Takeoff time'] = pd.Series(takeoff)
sondes_per_flight['Landing time'] = pd.Series(landing)
sondes_per_flight['Circles flown'] = pd.Series(circles_in_flight)
sondes_per_flight['Circles flown'].fillna(0,inplace=True)

# sondes_per_flight['Circles flown'].astype('int64',copy=False)
caption = 'Total dropsondes launched, circles flown during the flight, as well as takeoff and landing times (in UTC) for the flight are provided with corresponding flight IDs. The first part of the flight ID indicate the aircraft, and the latter part gives the flight date in the format MMDD'

print(sondes_per_flight.reset_index().to_latex(caption=caption,label='tab:flight_sondes',column_format='ccccc',index=False,float_format="%d",escape=False))

\begin{table}
\centering
\caption{Total dropsondes launched, circles flown during the flight, as well as takeoff and landing times (in UTC) for the flight are provided with corresponding flight IDs. The first part of the flight ID indicate the aircraft, and the latter part gives the flight date in the format MMDD}
\label{tab:flight_sondes}
\begin{tabular}{ccccc}
\toprule
 Flight ID &  Dropsondes launched &        Takeoff time &        Landing time &  Circles flown \\
\midrule
 HALO-0119 &                   15 & 2020-01-19 09:34:25 & 2020-01-19 18:48:03 &              1 \\
 HALO-0122 &                   73 & 2020-01-22 14:57:35 & 2020-01-23 00:10:30 &              6 \\
 HALO-0124 &                   77 & 2020-01-24 09:29:30 & 2020-01-24 18:41:13 &              6 \\
 HALO-0126 &                   75 & 2020-01-26 12:05:30 & 2020-01-26 21:20:49 &              6 \\
 HALO-0128 &                   74 & 2020-01-28 14:58:34 & 2020-01-28 23:55:17 &              6 \\
 HALO-0130 &                 

# Level-0 : Count of D-files

In [7]:
lv0_dfiles_halo = sorted(glob.glob(levels_dir[0]+'HALO/**/'+'D2020*_P*'))
lv0_dfiles_p3 = sorted(glob.glob(levels_dir[0]+'P3/**/'+'D2020*_P*'))

In [8]:
print(len(lv0_dfiles_p3))
print(len(lv0_dfiles_halo))


322
896


In [38]:
qc_HALO_file = glob.glob(levels_dir[2]+'logs_and_stats/Status_of_sondes_HALO_v0.8*.nc')
qc_P3_file = glob.glob(levels_dir[2]+'logs_and_stats/Status_of_sondes_P3_v0.8*.nc')

halo_stat = xr.open_dataset(qc_HALO_file[0])
p3_stat = xr.open_dataset(qc_P3_file[0])

qc = xr.concat([halo_stat,p3_stat],dim='time')

In [39]:
qc

<xarray.Dataset>
Dimensions:          (time: 1217)
Coordinates:
  * time             (time) datetime64[ns] 2020-01-19T16:14:10 ... 2020-02-11...
Data variables:
    s_time           (time) int64 3011 2955 2997 3152 ... 1060 1003 1021 1074
    s_t              (time) int64 1383 1385 1387 1474 1456 ... 498 494 499 505
    s_rh             (time) int64 1303 1318 1322 1404 1377 ... 502 498 494 508
    s_p              (time) int64 1381 1383 1385 1473 1455 ... 498 494 499 506
    s_z              (time) int64 274 1354 1347 1401 1390 ... 511 482 501 507
    s_u              (time) int64 31 2715 2730 2780 2866 ... 965 926 889 919 916
    s_v              (time) int64 31 2715 2730 2780 2866 ... 965 926 889 919 916
    s_alt            (time) int64 1381 1383 1385 1473 1455 ... 475 498 494 499 1
    ld_FLAG          (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    t_flag           (time) object 'good' 'good' 'good' ... 'good' 'good' 'good'
    rh_flag          (time) object 'good' 'good' 'good' ... 'good' 'good' 'good'
    p_flag           (time) object 'good' 'good' 'good' ... 'good' 'good' 'good'
    z_flag           (time) object 'ugly' 'good' 'good' ... 'good' 'good' 'good'
    u_flag           (time) object 'ugly' 'good' 'good' ... 'good' 'good' 'good'
    v_flag           (time) object 'ugly' 'good' 'good' ... 'good' 'good' 'good'
    alt_flag         (time) object 'good' 'good' 'good' ... 'good' 'good' 'ugly'
    srf_p_flag       (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    srf_z_flag       (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    srf_t_flag       (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    srf_rh_flag      (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    rms_palt_gpsalt  (time) float64 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 0.0
    ind_FLAG         (time) object 'UGLY' 'GOOD' 'GOOD' ... 'GOOD' 'GOOD' 'UGLY'
    srf_FLAG         (time) object 'GOOD' 'GOOD' 'GOOD' ... 'GOOD' 'GOOD' 'UGLY'
    FLAG             (time) object 'UGLY' 'GOOD' 'GOOD' ... 'GOOD' 'GOOD' 'UGLY'

In [46]:
qc.time.where((qc.ind_FLAG=='UGLY') & (qc.ld_FLAG == 0),drop=True)

<xarray.DataArray 'time' (time: 44)>
array(['2020-01-22T18:05:25.000000000', '2020-01-22T20:49:52.000000000',
       '2020-01-22T20:57:59.000000000', '2020-01-22T21:56:53.000000000',
       '2020-01-24T09:52:42.000000000', '2020-01-24T10:06:24.000000000',
       '2020-01-24T16:22:13.000000000', '2020-01-24T16:50:56.000000000',
       '2020-01-26T13:57:29.000000000', '2020-01-26T18:45:31.000000000',
       '2020-01-28T17:47:54.000000000', '2020-01-31T15:21:53.000000000',
       '2020-01-31T17:13:36.000000000', '2020-01-31T18:38:48.000000000',
       '2020-01-31T18:47:54.000000000', '2020-02-02T13:57:35.000000000',
       '2020-02-02T14:03:40.000000000', '2020-02-02T16:03:59.000000000',
       '2020-02-02T17:02:07.000000000', '2020-02-02T18:56:02.000000000',
       '2020-02-05T11:08:48.000000000', '2020-02-05T11:27:26.000000000',
       '2020-02-05T11:37:16.000000000', '2020-02-05T15:10:47.000000000',
       '2020-02-05T15:55:34.000000000', '2020-02-05T16:09:44.000000000',
       '2020-02-05T16:29:21.000000000', '2020-02-07T13:48:18.000000000',
       '2020-02-07T14:52:20.000000000', '2020-02-07T15:11:16.000000000',
       '2020-02-07T17:31:01.000000000', '2020-02-07T19:16:29.000000000',
       '2020-02-07T19:47:59.000000000', '2020-02-09T10:02:41.000000000',
       '2020-02-09T10:59:49.000000000', '2020-02-09T11:11:41.000000000',
       '2020-02-09T13:44:40.000000000', '2020-02-11T17:20:20.000000000',
       '2020-02-13T09:16:22.000000000', '2020-02-18T12:42:33.000000000',
       '2020-01-31T18:18:14.000000000', '2020-02-03T14:24:57.000000000',
       '2020-02-04T15:48:15.000000000', '2020-02-11T07:19:45.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-22T18:05:25 ... 2020-02-11T07:19:45